# Лабораторна робота №10
## Тема: Стиснення даних. Жадібна стратегія на прикладі кодування Гафмена 
### Виконала: Бояринцова Поліна
### Група: КН-24-1
### Дата: 22.06.2025

## 1. Реалізація прикладу алгоритму побудови дерева Гафмена

In [1]:
import heapq

# Вузол дерева Гафмена
class Node:
    def __init__(self, freq, symbol, left=None, right=None):
        self.freq = freq
        self.symbol = symbol
        self.left = left
        self.right = right
        self.huff = ''
    
    # Для порівняння в heapq
    def __lt__(self, other):
        return self.freq < other.freq

# Рекурсивний обхід дерева
def printNodes(node, val=''):
    newVal = val + str(node.huff)
    if node.left:
        printNodes(node.left, newVal)
    if node.right:
        printNodes(node.right, newVal)
    if not node.left and not node.right:
        print(f"{node.symbol} -> {newVal}")

# Дані
chars = ['a', 'b', 'c', 'd', 'e', 'f', 'g']
freq = [10, 7, 2, 2, 4, 4, 5]

# Побудова купи
nodes = []
for i in range(len(chars)):
    heapq.heappush(nodes, Node(freq[i], chars[i]))

while len(nodes) > 1:
    left = heapq.heappop(nodes)
    right = heapq.heappop(nodes)
    left.huff = 0
    right.huff = 1
    newNode = Node(left.freq + right.freq, left.symbol + right.symbol, left, right)
    heapq.heappush(nodes, newNode)

# Виведення кодів
printNodes(nodes[0])

b -> 00
f -> 010
c -> 0110
d -> 0111
e -> 100
g -> 101
a -> 11


## 2. Виконання завдання з наданим прикладом

### 2.1. Розв'язання завдання за алгоритмом Гафмена вручну

![розв'язання_вручну](images/1.jpg)

![розв'язання_вручну](images/2.jpg)

### 2.2. Перевірка результату за допомогою коду.

In [8]:
# A Вузол дерева Гафмена
import heapq
from collections import defaultdict

# для відображення кожного символу на його значення Гафмена
codes = {}

# Для зберігання частоти символів вхідних даних
freq = defaultdict(int)

# Вузол дерева Гафмена
class MinHeapNode:
    def __init__(self, data, freq):
        self.left = None
        self.right = None
        self.data = data
        self.freq = freq

    def __lt__(self, other):
        # Порівняння вузлів на основі їх частоти
        return self.freq < other.freq

# допоміжна функція для виведення символів разом з
# їх значеннями Гафмена
def printCodes(root, str_val):
    if root is None:
        return
    if root.data != '$': # '$' використовується для внутрішніх вузлів
        print(root.data, ":", str_val)
    printCodes(root.left, str_val + "0")
    printCodes(root.right, str_val + "1")

# допоміжна функція для зберігання символів разом з
# їх значеннями Гафмена в хеш-таблиці
def storeCodes(root, str_val):
    if root is None:
        return
    if root.data != '$':
        codes[root.data] = str_val
    storeCodes(root.left, str_val + "0")
    storeCodes(root.right, str_val + "1")

# функція для побудови дерева Гафмена та зберігання його
# в minHeap
def HuffmanCodes():
    global minHeap
    # Додаємо всі вузли символів до minHeap
    for key in freq:
        heapq.heappush(minHeap, MinHeapNode(key, freq[key]))

    # Будуємо дерево Гафмена
    while len(minHeap) != 1:
        # Витягуємо два вузли з найменшою частотою
        left = heapq.heappop(minHeap)
        right = heapq.heappop(minHeap)

        # Створюємо новий внутрішній вузол
        top = MinHeapNode('$', left.freq + right.freq)
        top.left = left
        top.right = right
        heapq.heappush(minHeap, top)
    
    # Зберігаємо коди Гафмена після побудови дерева
    storeCodes(minHeap[0], "")

# допоміжна функція для відображення кожного символу на його
# частоту у вхідному рядку
def calcFreq(input_str):
    for char in input_str:
        freq[char] += 1

# функція перебирає закодований рядок s
# якщо s[i]=='1', то переходить до node->right
# якщо s[i]=='0', то переходить до node->left
# якщо це листовий вузол, додає node->data до нашого вихідного рядка
def decode_file(root, s):
    ans = ""
    curr = root
    n = len(s)
    for i in range(n):
        if s[i] == '0':
            curr = curr.left
        else:
            curr = curr.right

        # досягнуто листового вузла
        if curr.left is None and curr.right is None:
            ans += curr.data
            curr = root # Повертаємося до кореня для наступного символу
    return ans

# Головний код
if __name__ == "__main__":
    minHeap = []
    # Вхідний рядок для обробки
    input_str = "ABABCAADEEGGFGFABBABAABCBAADEEGFGF" # Використовуємо рядок з попереднього запиту

    # Обчислюємо частоту символів
    calcFreq(input_str)

    # Будуємо коди Гафмена
    HuffmanCodes()

    print("Частоти символів:")
    for char, frequency in sorted(freq.items()):
        print(f"'{char}': {frequency}")

    print("\nКоди Гафмена:")
    for key in sorted(codes):
        print(key, ":", codes[key])

    # Кодуємо вхідний рядок
    encodedString = ""
    for char in input_str:
        encodedString += codes[char]

    print("\nЗакодовані дані Гафмена:")
    print(encodedString)

    # Декодуємо закодований рядок
    decodedString = decode_file(minHeap[0], encodedString)
    print("\nДекодовані дані Гафмена:")
    print(decodedString)

    # Перевірка, чи збігається декодований рядок з оригіналом
    print(f"\nДекодований рядок збігається з оригіналом: {decodedString == input_str}")

Частоти символів:
'A': 10
'B': 7
'C': 2
'D': 2
'E': 4
'F': 4
'G': 5

Коди Гафмена:
A : 11
B : 00
C : 0110
D : 0111
E : 100
F : 010
G : 101

Закодовані дані Гафмена:
11001100011011110111100100101101010101010110000110011110001100011110111100100101010101010

Декодовані дані Гафмена:
ABABCAADEEGGFGFABBABAABCBAADEEGFGF

Декодований рядок збігається з оригіналом: True


>Як можна побачити, вручну розв'язання *не дійшло до глобального оптимуму.* Є краще рішення. Спробуємо його знайти.

![краще_розв'язання](images/3.jpg)

## 3. Контрольні питання

### 1. Що таке жадібні алгоритми?

Жадібні алгоритми — це методи розв’язання задач, у яких на кожному кроці вибирається локально оптимальне рішення (тобто найкраще на даний момент) з надією, що така стратегія приведе до глобального оптимуму.

### 2. Що таке префіксний код? Який код використовується у коді Гафмена? 

Префіксний код — це такий код, у якому жодне закодоване значення не є префіксом іншого.
Це дозволяє однозначно декодувати будь-яку послідовність без роздільників.

Код Гафмена — це префіксний двійковий код, який базується на частоті символів: символи з більшою частотою мають коротші коди, а з меншою — довші.

### 3. Як пов’язана структура даних «купа» зі структурою даних «черга з пріоритетами»? 

**Купа (heap)** — це структура даних на основі бінарного дерева, в якій батьківський елемент має більший або менший пріоритет, ніж його нащадки.  
**Черга з пріоритетами** — це абстракція, яка дозволяє швидко витягати елемент із найвищим (або найнижчим) пріоритетом.  
*Зв’язок:* Купа є ефективною реалізацією черги з пріоритетами.  
Наприклад, мін-купа дозволяє за O(logn) додавати елементи і за O(logn) витягати найменший.

### 4. Що таке стиснення даних і для чого воно використовується? Які його основні переваги? 

Стиснення даних — це процес перетворення даних у форму, що займає менше пам’яті або обсягу для зберігання чи передавання.
Основні переваги:

- Зменшення обсягу пам’яті.
- Прискорення передачі даних по мережі.
- Зменшення вартості зберігання або обслуговування.
- Підвищення ефективності архівування та резервного копіювання.

### 5. Які кроки необхідно виконати для стиснення даних за допомогою алгоритму кодування Гафмена? 

1. Обчислити частоти всіх символів у тексті.
2. Створити вузли для кожного символу з вагою (частотою).
3. Помістити вузли в мін-купу (чергу з пріоритетами).
4. Побудувати дерево Гафмена, об’єднуючи найменші вузли.
5. Присвоїти коди кожному символу, проходячи по дереву (0 — вліво, 1 — вправо).
6. Закодувати текст, замінюючи символи на їхні коди.
7. (Опціонально) Зберегти дерево або таблицю кодів для декодування.

### 6. Які основні обмеження та недоліки алгоритму кодування Гафмена? Чи можливо покращити його продуктивність? 

**Обмеження:**  

- Ефективність зменшується, якщо частоти символів мало відрізняються.  
- Не підходить для потокових даних без попереднього аналізу частот.  
- Не оптимальний для невеликих повідомлень.   
**Можливі покращення:**
  
- Використання адаптивного Гафмена (оновлює дерево на ходу).  
- Комбінування з іншими методами.  

### 7. Які існують альтернативні методи стиснення даних, що можуть конкурувати з алгоритмом Гафмена? 

Альтернативи:

- Арифметичне кодування — часто ефективніше за Гафмена.  
- LZ77 / LZ78 / LZW — використовуються в ZIP, GIF.  
- RLE (Run-Length Encoding) — ефективне для повторюваних символів.  
- DEFLATE (комбінація LZ77 + Huffman) — основа ZIP, PNG.  

### 8. Які практичні застосування можуть мати алгоритми стиснення даних, зокрема алгоритм Гафмена, у сучасних інформаційних системах? 

Застосування:

- Формати файлів: ZIP, PNG, MP3 (як частина алгоритму).  
- Архіватори: WinRAR, 7-Zip, gzip (частково Гафмен).  
- Передача даних по мережі.  
- Операційні системи: для стискання логів, резервних копій.  
- Інформаційна безпека: зменшення розміру перед шифруванням.  
- Мобільні пристрої: економія пам’яті та трафіку.  